# 코랩 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# 데이터 전처리 목차

####1. 데이터 준비
- 데이터 불러오기
- 분석 대상 데이터 선택
- 데이터프레임 생성

####2. 전처리
- 전처리: context에서 공백, 영어(알파벳), 특수문자( _ 제외), 줄바꿈, 탭, 단독 숫자, 숫자, 한자, 연속된 띄어쓰기 제거 
- 빈 문자열을 null값으로 replcae하고 포함 행 제거 후, 인덱스 재정렬(context, code, context_clean 순서로) 
- 라벨 인코딩
- 모델 입력 토큰 수에 맞춘 데이터 길이 조정
- 각 라벨당 동일한 개수로 통일(ex:100개)

####3. 데이터 분할 및 저장
- 데이터셋 분할 (Train/Test)
- 데이터 저장

# 데이터 준비

###데이터 불러오기

In [ ]:
import pandas as pd #데이터 처리 및 분석 라이브러리
import json #json 파일 다루기 위한 라이브러리

In [ ]:
with open('/content/drive/MyDrive/1_BERT code-20250203T043054Z-001/1_BERT code/data/raw_data_news.json') as f:
    raw_js = json.loads(f.read())

In [ ]:
raw_js

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
raw_data = raw_js["data"]
#raw_js에서 data라는 key에 해당하는 값을 가져옴

In [ ]:
raw_data

Output hidden; open in https://colab.research.google.com to view.

### 분석 대상 데이터 찾기

In [ ]:
code_list = [entry['doc_class']['code'] for entry in raw_data]

In [ ]:
code_list

['정치',
 '정치',
 '정치',
 '사회',
 '문화',
 '사회',
 '사회',
 '경제',
 '사회',
 '사회',
 '사회',
 '정치',
 '사회',
 '사회',
 '국제',
 '국제',
 '사회',
 '사회',
 '스포츠',
 '경제',
 '경제',
 '정치',
 '사회',
 '경제',
 '문화',
 '사회',
 '경제',
 '문화',
 '사회',
 '사회',
 '사회',
 '정치',
 '스포츠',
 '경제',
 '경제',
 '국제',
 '사회',
 '국제',
 '사회',
 'IT과학',
 'IT과학',
 '사회',
 '정치',
 '사회',
 '정치',
 '스포츠',
 '문화',
 '경제',
 '스포츠',
 '사회',
 '사회',
 '사회',
 '사회',
 '경제',
 '사회',
 '사회',
 '사회',
 '경제',
 '사회',
 '사회',
 '정치',
 '사회',
 '경제',
 '정치',
 '사회',
 '사회',
 '사회',
 '경제',
 '국제',
 '사회',
 '정치',
 '정치',
 '사회',
 '사회',
 '국제',
 '정치',
 '정치',
 '정치',
 '정치',
 '사회',
 '정치',
 '사회',
 '문화',
 '사회',
 '정치',
 '사회',
 'IT과학',
 '사회',
 '정치',
 '사회',
 '정치',
 'IT과학',
 '정치',
 '사회',
 '사회',
 '사회',
 '사회',
 '문화',
 '사회',
 '사회',
 '사회',
 '사회',
 '사회',
 '정치',
 '정치',
 '경제',
 '문화',
 '사회',
 '사회',
 '사회',
 '국제',
 '정치',
 '사회',
 '사회',
 '사회',
 '경제',
 '정치',
 '사회',
 '사회',
 'IT과학',
 '사회',
 '경제',
 '정치',
 '사회',
 '사회',
 '사회',
 '정치',
 '사회',
 '국제',
 '정치',
 '문화',
 '문화',
 '사회',
 '사회',
 '사회',
 '스포츠',
 '정치',
 'IT과학',
 '경제',
 '사회',
 '경

In [ ]:
context_list = [entry['paragraphs'][0]['context'] for entry in raw_data]

In [ ]:
context_list

['2022년 대통령 선거를 진두지휘할 여당의 신임 당대표에 5선 송영길 의원이 당선됐다. 함께 치러진 최고위원 선거에서는 김용민 강병원 백혜련 김영배 전혜숙 의원이 당원들의 선택을 받았다. 더불어민주당 당원들은 당대표 선거에서는 ‘쇄신’을 강조한 송 의원에게, 최고위원 선거에서는 문재인정부의 성공적인 마무리를 위한 ‘친문(친문재인) 지도부’ 구성에 힘을 실어준 것으로 평가된다. \n \n2일 서울 여의도 당사에서 열린 민주당 임시 전국대의원회의에서 송 신임 대표는 총득표율 35.60%로 홍영표 의원을 불과 0.59%포인트 차로 누르고 당권을 손에 쥐었다. 2위 홍 의원은 권리당원 투표에서 송 의원을 앞섰지만, 반영율이 더 큰 대의원 투표에서 밀렸다. \n \n송 신임 대표는 전당대회 직후 수락연설에서 “유능한 개혁, 언행일치의 민주당을 만들어 국민의 삶을 지켜내고, 국민의 마음을 얻겠다”며 “자랑스런 대선 주자들과 소통하면서 대선 경선을 공정하게 관리하겠다”고 강조했다. \n \n \n최고위원 선거에서는 민주당 초선모임 ‘처럼회’ 멤버로 중대범죄수사청 설치 등을 주장해 온 강성 친문 김용민 의원이 가장 많은 표를 얻었다. 역시 친문 진영으로 분류되는 강병원(재선) 김영배(초선) 전혜숙(3선) 의원이 지도부에 입성했다. 비주류 진영은 ‘조국 사태’ 등에 대해 쓴소리를 했던 재선의 백혜련 의원이 유일하다. \n \n송 신임 대표가 ‘범친문’으로 분류되긴 하지만 이번 전대기간 당의 전면적 쇄신을 요구해왔고, 친문 강성 지지층에 대한 쓴소리도 아끼지 않았다는 점에서 당내 긴장 상황이 조성될 수 있다는 우려가 일각에서 나온다. 특히 친문 핵심인 윤호중 원내대표는 물론 선출직 최고위원 5명 중 4명이 친문 진영인 상황에서 이들과 얼마나 호흡을 맞출 수 있느냐도 정치권의 관심사다. \n \n1963년 전남 고흥에서 태어난 송 신임 대표는 인천 계양을에서 16대, 17대, 18대, 20대, 21대 국회의원을 지냈다. 2010년부터 4년간 인천시장을 역임했다. \n ',
 

### context와 label로 데이터 프레임으로 만들기

In [ ]:
df_v1 = pd.DataFrame({'code': code_list, 'context': context_list})

In [ ]:
df_v1

,code,context
0,정치,2022년 대통령 선거를 진두지휘할 여당의 신임 당대표에 5선 송영길 의원이 당선됐...
1,정치,북한이 2일 대남·대미 담화 3건을 동시다발적으로 발표하면서 남측과 미국을 맹비난했...
2,정치,서울시는 코로나19 완치자의 빠른 일상 회복을 돕기 위해 4주간 건강관리서비스를 지...
3,사회,전남도소방본부가 전국 최초로 사회적·경제적 취약계층으로 찾아가는 ‘119생활안전전담...
4,문화,여성가족부가 지난달 27일 ‘제4차 건강가정기본계획(2021~2025)’을 통해 난...
...,...,...
24004,사회,연세대학교 신촌캠퍼스에 위치한 연세우유 서울사무소에서 14일 코로나19 확진자가 나...
24005,사회,"아베 신조 일본 총리가 최근 현지에서 논란을 빚은 '검찰청법 개정안' 관련 재차 ""..."
24006,IT과학,서울사이버대학교는 K-MOOC 플랫폼을 통해 5개 과정 강좌를 무료로 제공한다고 1...
24007,사회,SK에너지가 모빌리티 고객을 대상으로 한 차량관리 통합 서비스 개발에 본격 착수했다...


In [ ]:
df_v1["code"].value_counts()  #code 컬럼의 각 값이 몇 번 등장하는지 개수 계산 (카테고리별 개수 계산)

사회      7825
경제      5931
정치      2643
문화      2320
국제      2012
스포츠     1184
IT과학    1015
기타       653
지역       426
Name: code, dtype: int64

In [ ]:
# 특정 카테고리 ("기타", "지역") 삭제
labels_to_remove = ['기타', '지역']
df_v2 = df_v1[~df_v1['code'].isin(labels_to_remove)]

In [ ]:
df_v2["code"].value_counts()

,count
code,
사회,7825
경제,5931
정치,2643
문화,2320
국제,2012
스포츠,1184
IT과학,1015


# 전처리

In [ ]:
import re
import numpy as np

### ???

In [ ]:
# 전처리
def cleantext(text):
    text = text.strip()   #문자열 앞뒤 공백 제거
    text = re.sub(r"[a-zA-Z]+", "", text)   # 영어(알파벳 대소문자) 삭제
    text = re.sub(r"[^\w\s_]", "", text)   # 특수문자 제거 (단 _는 남김)
    text = re.sub('\n', ' ', text)  # 줄바꿈 제거 후 공백 대체
    text = re.sub('\t', ' ', text)  # 탭 제거 후 공백 대체
    text = re.sub(r"\b\d+\b", "", text) # 단독 숫자 제거
    text = re.sub(r"\d+", "", text) # 숫자 제거 (위에서 처리되지 않은 연속 숫자 제거)
    text = re.sub('[\u4e00-\u9fff]+', '', text) # 한자 삭제
    text = re.sub(r" +", ' ', text) # 연속된 띄어쓰기 제거
    text = text.strip()
    return text

In [ ]:
# 새로운 열 생성(전처리한 문장)
df_v2['context_clean'] = df_v2['context'].apply(lambda x : cleantext(x))

<ipython-input-34-b908df3764aa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_v2['context_clean'] = df_v2['context'].apply(lambda x : cleantext(x))


In [ ]:
df_v2

,code,context,context_clean
0,정치,2022년 대통령 선거를 진두지휘할 여당의 신임 당대표에 5선 송영길 의원이 당선됐...,년 대통령 선거를 진두지휘할 여당의 신임 당대표에 선 송영길 의원이 당선됐다 함께 ...
1,정치,북한이 2일 대남·대미 담화 3건을 동시다발적으로 발표하면서 남측과 미국을 맹비난했...,북한이 일 대남대미 담화 건을 동시다발적으로 발표하면서 남측과 미국을 맹비난했다 북...
2,정치,서울시는 코로나19 완치자의 빠른 일상 회복을 돕기 위해 4주간 건강관리서비스를 지...,서울시는 코로나 완치자의 빠른 일상 회복을 돕기 위해 주간 건강관리서비스를 지원하고...
3,사회,전남도소방본부가 전국 최초로 사회적·경제적 취약계층으로 찾아가는 ‘119생활안전전담...,전남도소방본부가 전국 최초로 사회적경제적 취약계층으로 찾아가는 생활안전전담대를 운영...
4,문화,여성가족부가 지난달 27일 ‘제4차 건강가정기본계획(2021~2025)’을 통해 난...,여성가족부가 지난달 일 제차 건강가정기본계획을 통해 난자정자 공여나 대리출산 등으로...
...,...,...,...
24004,사회,연세대학교 신촌캠퍼스에 위치한 연세우유 서울사무소에서 14일 코로나19 확진자가 나...,연세대학교 신촌캠퍼스에 위치한 연세우유 서울사무소에서 일 코로나 확진자가 나왔다 확...
24005,사회,"아베 신조 일본 총리가 최근 현지에서 논란을 빚은 '검찰청법 개정안' 관련 재차 ""...",아베 신조 일본 총리가 최근 현지에서 논란을 빚은 검찰청법 개정안 관련 재차 자의적...
24006,IT과학,서울사이버대학교는 K-MOOC 플랫폼을 통해 5개 과정 강좌를 무료로 제공한다고 1...,서울사이버대학교는 플랫폼을 통해 개 과정 강좌를 무료로 제공한다고 일 밝혔다 강좌는...
24007,사회,SK에너지가 모빌리티 고객을 대상으로 한 차량관리 통합 서비스 개발에 본격 착수했다...,에너지가 모빌리티 고객을 대상으로 한 차량관리 통합 서비스 개발에 본격 착수했다 에...


### ???

In [ ]:
# 빈 문자열 null값(NaN)으로 replace하고 포함 행 제거, 인덱스 재정렬
df_v2 = df_v2.replace('', np.nan).dropna(subset=['context_clean']).reset_index(drop=True)

In [ ]:
print(df_v2.isnull().all())

code             False
context          False
context_clean    False
dtype: bool


In [ ]:
# 열 순서 변경
df_v2 = df_v2[["context", "code", "context_clean"]]

In [ ]:
df_v2

,context,code,context_clean
0,2022년 대통령 선거를 진두지휘할 여당의 신임 당대표에 5선 송영길 의원이 당선됐...,정치,년 대통령 선거를 진두지휘할 여당의 신임 당대표에 선 송영길 의원이 당선됐다 함께 ...
1,북한이 2일 대남·대미 담화 3건을 동시다발적으로 발표하면서 남측과 미국을 맹비난했...,정치,북한이 일 대남대미 담화 건을 동시다발적으로 발표하면서 남측과 미국을 맹비난했다 북...
2,서울시는 코로나19 완치자의 빠른 일상 회복을 돕기 위해 4주간 건강관리서비스를 지...,정치,서울시는 코로나 완치자의 빠른 일상 회복을 돕기 위해 주간 건강관리서비스를 지원하고...
3,전남도소방본부가 전국 최초로 사회적·경제적 취약계층으로 찾아가는 ‘119생활안전전담...,사회,전남도소방본부가 전국 최초로 사회적경제적 취약계층으로 찾아가는 생활안전전담대를 운영...
4,여성가족부가 지난달 27일 ‘제4차 건강가정기본계획(2021~2025)’을 통해 난...,문화,여성가족부가 지난달 일 제차 건강가정기본계획을 통해 난자정자 공여나 대리출산 등으로...
...,...,...,...
22925,연세대학교 신촌캠퍼스에 위치한 연세우유 서울사무소에서 14일 코로나19 확진자가 나...,사회,연세대학교 신촌캠퍼스에 위치한 연세우유 서울사무소에서 일 코로나 확진자가 나왔다 확...
22926,"아베 신조 일본 총리가 최근 현지에서 논란을 빚은 '검찰청법 개정안' 관련 재차 ""...",사회,아베 신조 일본 총리가 최근 현지에서 논란을 빚은 검찰청법 개정안 관련 재차 자의적...
22927,서울사이버대학교는 K-MOOC 플랫폼을 통해 5개 과정 강좌를 무료로 제공한다고 1...,IT과학,서울사이버대학교는 플랫폼을 통해 개 과정 강좌를 무료로 제공한다고 일 밝혔다 강좌는...
22928,SK에너지가 모빌리티 고객을 대상으로 한 차량관리 통합 서비스 개발에 본격 착수했다...,사회,에너지가 모빌리티 고객을 대상으로 한 차량관리 통합 서비스 개발에 본격 착수했다 에...


### 라벨 인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder #범주형 변수를 숫자로 변환하기 유용

In [ ]:
labelencoder = LabelEncoder()
df_v2['label'] = (labelencoder.fit_transform(df_v2["code"])).astype('int64')
#code 컬럼 (카테고리)을 숫자로 변환해 label이라는 새로운 열 생성

In [ ]:
# 인코딩 현황을 딕셔너리(카테고리:key, 인코딩된 숫자:value)로 확인
encoding_mapping = dict(zip(labelencoder.classes_, labelencoder.transform(labelencoder.classes_)))
print(encoding_mapping)

{'IT과학': 0, '경제': 1, '국제': 2, '문화': 3, '사회': 4, '스포츠': 5, '정치': 6}


In [ ]:
df_v2

,context,code,context_clean,label
0,2022년 대통령 선거를 진두지휘할 여당의 신임 당대표에 5선 송영길 의원이 당선됐...,정치,년 대통령 선거를 진두지휘할 여당의 신임 당대표에 선 송영길 의원이 당선됐다 함께 ...,6
1,북한이 2일 대남·대미 담화 3건을 동시다발적으로 발표하면서 남측과 미국을 맹비난했...,정치,북한이 일 대남대미 담화 건을 동시다발적으로 발표하면서 남측과 미국을 맹비난했다 북...,6
2,서울시는 코로나19 완치자의 빠른 일상 회복을 돕기 위해 4주간 건강관리서비스를 지...,정치,서울시는 코로나 완치자의 빠른 일상 회복을 돕기 위해 주간 건강관리서비스를 지원하고...,6
3,전남도소방본부가 전국 최초로 사회적·경제적 취약계층으로 찾아가는 ‘119생활안전전담...,사회,전남도소방본부가 전국 최초로 사회적경제적 취약계층으로 찾아가는 생활안전전담대를 운영...,4
4,여성가족부가 지난달 27일 ‘제4차 건강가정기본계획(2021~2025)’을 통해 난...,문화,여성가족부가 지난달 일 제차 건강가정기본계획을 통해 난자정자 공여나 대리출산 등으로...,3
...,...,...,...,...
22925,연세대학교 신촌캠퍼스에 위치한 연세우유 서울사무소에서 14일 코로나19 확진자가 나...,사회,연세대학교 신촌캠퍼스에 위치한 연세우유 서울사무소에서 일 코로나 확진자가 나왔다 확...,4
22926,"아베 신조 일본 총리가 최근 현지에서 논란을 빚은 '검찰청법 개정안' 관련 재차 ""...",사회,아베 신조 일본 총리가 최근 현지에서 논란을 빚은 검찰청법 개정안 관련 재차 자의적...,4
22927,서울사이버대학교는 K-MOOC 플랫폼을 통해 5개 과정 강좌를 무료로 제공한다고 1...,IT과학,서울사이버대학교는 플랫폼을 통해 개 과정 강좌를 무료로 제공한다고 일 밝혔다 강좌는...,0
22928,SK에너지가 모빌리티 고객을 대상으로 한 차량관리 통합 서비스 개발에 본격 착수했다...,사회,에너지가 모빌리티 고객을 대상으로 한 차량관리 통합 서비스 개발에 본격 착수했다 에...,4


# 데이터 셋 만들기

### 데이터셋 길이 조정

In [ ]:
# 데이터셋 길이 확인 (버트는 최대 512토큰)
def calculate_lengths(df, column_name):

    column = df[column_name]     #데이터프레임에서 특정 열 선택

    lengths = column.str.len()   #str.len():  각 행의 문자열 길이 계산

    min_length = lengths.min()   #최소 길이
    max_length = lengths.max()   #최대 길이
    average_length = lengths.mean()  #평균 길이

    return min_length, max_length, average_length

In [ ]:
min, max, average = calculate_lengths(df_v2, "context_clean")  #df_v2데이터프레임 context_clean 열의 각 문장 길이 정보 계산
print(f"min:{min}, max:{max}, average:{average}")

min:135, max:1910, average:980.0453118185783


In [ ]:
df_v2['context_clean'].str.len()

,context_clean
0,878
1,1492
2,900
3,678
4,1784
...,...
22925,342
22926,443
22927,901
22928,806


In [ ]:
# 데이터 길이 지정
df_v3 = df_v2[df_v2['context_clean'].str.len().between(0, 1500)].reset_index(drop=True)

In [ ]:
df_v3

,context,code,context_clean,label
0,2022년 대통령 선거를 진두지휘할 여당의 신임 당대표에 5선 송영길 의원이 당선됐...,정치,년 대통령 선거를 진두지휘할 여당의 신임 당대표에 선 송영길 의원이 당선됐다 함께 ...,6
1,북한이 2일 대남·대미 담화 3건을 동시다발적으로 발표하면서 남측과 미국을 맹비난했...,정치,북한이 일 대남대미 담화 건을 동시다발적으로 발표하면서 남측과 미국을 맹비난했다 북...,6
2,서울시는 코로나19 완치자의 빠른 일상 회복을 돕기 위해 4주간 건강관리서비스를 지...,정치,서울시는 코로나 완치자의 빠른 일상 회복을 돕기 위해 주간 건강관리서비스를 지원하고...,6
3,전남도소방본부가 전국 최초로 사회적·경제적 취약계층으로 찾아가는 ‘119생활안전전담...,사회,전남도소방본부가 전국 최초로 사회적경제적 취약계층으로 찾아가는 생활안전전담대를 운영...,4
4,5월 코로나19 백신 접종이 심상치 않다. 물량 부족으로 화이자 백신이 1차 접종을...,사회,월 코로나 백신 접종이 심상치 않다 물량 부족으로 화이자 백신이 차 접종을 일시 중...,4
...,...,...,...,...
19145,연세대학교 신촌캠퍼스에 위치한 연세우유 서울사무소에서 14일 코로나19 확진자가 나...,사회,연세대학교 신촌캠퍼스에 위치한 연세우유 서울사무소에서 일 코로나 확진자가 나왔다 확...,4
19146,"아베 신조 일본 총리가 최근 현지에서 논란을 빚은 '검찰청법 개정안' 관련 재차 ""...",사회,아베 신조 일본 총리가 최근 현지에서 논란을 빚은 검찰청법 개정안 관련 재차 자의적...,4
19147,서울사이버대학교는 K-MOOC 플랫폼을 통해 5개 과정 강좌를 무료로 제공한다고 1...,IT과학,서울사이버대학교는 플랫폼을 통해 개 과정 강좌를 무료로 제공한다고 일 밝혔다 강좌는...,0
19148,SK에너지가 모빌리티 고객을 대상으로 한 차량관리 통합 서비스 개발에 본격 착수했다...,사회,에너지가 모빌리티 고객을 대상으로 한 차량관리 통합 서비스 개발에 본격 착수했다 에...,4


### ???

In [ ]:
df_v3["label"].value_counts()

,count
label,
4,7539
1,4163
6,2139
3,1930
2,1566
5,1013
0,800


In [ ]:
# 라벨 확인
label_list = df_v3["label"].unique().tolist()
print(label_list)

[6, 4, 1, 2, 5, 3, 0]


In [ ]:
def sample_data_by_label(dataframe, label_column, label_values, num_samples, random_state):
    sampled_data = pd.concat([dataframe[dataframe[label_column] == label].sample(n=num_samples, random_state=random_state)
                             for label in label_values], ignore_index=True)
    return sampled_data

In [ ]:
df_v4 = sample_data_by_label(df_v3, 'label', label_list, 100, random_state=42)

In [ ]:
df_v4["label"].value_counts()

,count
label,
6,100
4,100
1,100
2,100
5,100
3,100
0,100


# Input/Output Data 만들기

In [ ]:
from sklearn.model_selection import train_test_split #데이터 분할(train/test)에 유용

In [ ]:
# 전처리한 data를 input으로(type = pandas.core.series.Series)
input_data = df_v4['context_clean']

# label을 output으로(type = pandas.core.series.Series)
output_data = df_v4["label"]

## 데이터 나누기

In [ ]:
# train : test = 8 : 2
train_input, test_input, train_output, test_output = train_test_split(input_data,
                                                                      output_data,
                                                                      test_size = 0.2, #test 데이터 20%
                                                                      shuffle = True,  #데이터 무작위 섞어 train/test 랜덤 분
                                                                      stratify= output_data, #각 라벨이 균등하게 분포
                                                                      random_state = 42)

# test/val은 모델편에서 나눌 예정

In [ ]:
# 최종 데이터 셋 train : test = 8 : 2
print(len(train_input))
print(len(test_input))

560
140


## 데이터 저장

In [ ]:
train_input.to_pickle("/content/drive/MyDrive/멘토링/data/v1/train_input.pickle")
train_output.to_pickle("/content/drive/MyDrive/멘토링/data/v1/train_output.pickle")
test_input.to_pickle("/content/drive/MyDrive/멘토링/data/v1/test_input.pickle")
test_output.to_pickle("/content/drive/MyDrive/멘토링/data/v1/test_output.pickle")

In [ ]:
# 하나의 데이터프레임으로 만드려면
# train_data = pd.concat([train_input, train_output],axis=1)
# test_data = pd.concat([test_input, test_output],axis=1)

In [ ]:
# train_data.to_pickle(r"C:/Users/user/Desktop/2024mtr/data/v1/train_data.pickle")
# test_data.to_pickle(r"C:/Users/user/Desktop/2024mtr/data/v1/test_data.pickle")